# Multi-Layer Perceptron Model

Train a 3-layer neural network using PyTorch with early stopping for loan default prediction.

In [1]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    brier_score_loss,
    classification_report,
    confusion_matrix
)

# Configuration
DATA_DIR = "../data"
MODELS_DIR = "../models"
RESULTS_DIR = "../results"

TRAIN_FILE = os.path.join(DATA_DIR, "train.csv")
VAL_FILE = os.path.join(DATA_DIR, "val.csv")
TEST_FILE = os.path.join(DATA_DIR, "test.csv")

MODEL_FILE = os.path.join(MODELS_DIR, "mlp_model.pth")
PREDICTIONS_FILE = os.path.join(RESULTS_DIR, "mlp_predictions.csv")
METRICS_FILE = os.path.join(RESULTS_DIR, "mlp_metrics.json")

TARGET_COL = "status"
RANDOM_STATE = 42

BATCH_SIZE = 512
MAX_EPOCHS = 50
LEARNING_RATE = 0.001
PATIENCE = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

In [2]:
# Define MLP architecture
class SimpleMLP(nn.Module):
    def __init__(self, input_dim):
        super(SimpleMLP, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

In [3]:
# Load data
train_df = pd.read_csv(TRAIN_FILE)
val_df = pd.read_csv(VAL_FILE)
test_df = pd.read_csv(TEST_FILE)

X_train = train_df.drop(columns=[TARGET_COL]).values
y_train = train_df[TARGET_COL].values

X_val = val_df.drop(columns=[TARGET_COL]).values
y_val = val_df[TARGET_COL].values

X_test = test_df.drop(columns=[TARGET_COL]).values
y_test = test_df[TARGET_COL].values

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

Training set: (179250, 67)
Validation set: (14867, 67)
Test set: (14867, 67)


In [4]:
# Create dataloaders
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
# Initialize model
input_dim = X_train.shape[1]
model = SimpleMLP(input_dim).to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
print(f"Training on device: {DEVICE}")

Model initialized with 19073 parameters
Training on device: cpu


In [6]:
# Training functions
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
    
    return total_loss / len(val_loader)

In [7]:
# Train with early stopping
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(MAX_EPOCHS):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss = validate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{MAX_EPOCHS} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
    
    if patience_counter >= PATIENCE:
        print(f"Early stopping triggered after {epoch+1} epochs")
        break

model.load_state_dict(best_model_state)
print(f"Best validation loss: {best_val_loss:.4f}")

Epoch 1/50 - Train Loss: 0.4768, Val Loss: 0.3757
Epoch 2/50 - Train Loss: 0.4201, Val Loss: 0.3573
Epoch 3/50 - Train Loss: 0.4084, Val Loss: 0.3561
Epoch 4/50 - Train Loss: 0.4004, Val Loss: 0.3705
Epoch 5/50 - Train Loss: 0.3942, Val Loss: 0.3666
Epoch 6/50 - Train Loss: 0.3901, Val Loss: 0.3561
Epoch 7/50 - Train Loss: 0.3860, Val Loss: 0.3464
Epoch 8/50 - Train Loss: 0.3821, Val Loss: 0.3314
Epoch 9/50 - Train Loss: 0.3807, Val Loss: 0.3378
Epoch 10/50 - Train Loss: 0.3768, Val Loss: 0.3560
Epoch 11/50 - Train Loss: 0.3729, Val Loss: 0.3347
Epoch 12/50 - Train Loss: 0.3717, Val Loss: 0.3477
Epoch 13/50 - Train Loss: 0.3680, Val Loss: 0.3284
Epoch 14/50 - Train Loss: 0.3678, Val Loss: 0.3410
Epoch 15/50 - Train Loss: 0.3637, Val Loss: 0.3339
Epoch 16/50 - Train Loss: 0.3618, Val Loss: 0.3377
Epoch 17/50 - Train Loss: 0.3615, Val Loss: 0.3383
Epoch 18/50 - Train Loss: 0.3597, Val Loss: 0.3598
Early stopping triggered after 18 epochs
Best validation loss: 0.3284


In [8]:
# Evaluate on validation set
model.eval()
X_val_tensor = torch.FloatTensor(X_val).to(DEVICE)

with torch.no_grad():
    y_val_proba = model(X_val_tensor).cpu().numpy().flatten()

y_val_pred = (y_val_proba >= 0.5).astype(int)

val_auc_roc = roc_auc_score(y_val, y_val_proba)
val_auc_pr = average_precision_score(y_val, y_val_proba)
val_brier = brier_score_loss(y_val, y_val_proba)

print("Validation Set Performance:")
print(f"AUC-ROC: {val_auc_roc:.4f}")
print(f"AUC-PR: {val_auc_pr:.4f}")
print(f"Brier Score: {val_brier:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

Validation Set Performance:
AUC-ROC: 0.8815
AUC-PR: 0.8295
Brier Score: 0.1101

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.91     11203
           1       0.71      0.73      0.72      3664

    accuracy                           0.86     14867
   macro avg       0.81      0.81      0.81     14867
weighted avg       0.86      0.86      0.86     14867


Confusion Matrix:
[[10109  1094]
 [ 1006  2658]]


In [9]:
# Evaluate on test set
X_test_tensor = torch.FloatTensor(X_test).to(DEVICE)

with torch.no_grad():
    y_test_proba = model(X_test_tensor).cpu().numpy().flatten()

y_test_pred = (y_test_proba >= 0.5).astype(int)

test_auc_roc = roc_auc_score(y_test, y_test_proba)
test_auc_pr = average_precision_score(y_test, y_test_proba)
test_brier = brier_score_loss(y_test, y_test_proba)

print("Test Set Performance:")
print(f"AUC-ROC: {test_auc_roc:.4f}")
print(f"AUC-PR: {test_auc_pr:.4f}")
print(f"Brier Score: {test_brier:.4f}")

Test Set Performance:
AUC-ROC: 0.8917
AUC-PR: 0.8412
Brier Score: 0.1075


In [10]:
# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'input_dim': input_dim
}, MODEL_FILE)
print(f"Model saved to {MODEL_FILE}")

# Save predictions
predictions_df = pd.DataFrame({
    'true_label': y_val,
    'predicted_probability': y_val_proba,
    'predicted_label': y_val_pred,
    'dataset': 'validation'
})
predictions_df.to_csv(PREDICTIONS_FILE, index=False)
print(f"Predictions saved to {PREDICTIONS_FILE}")

# Save metrics
all_metrics = {
    'architecture': 'Input → 128 → 64 → 32 → 1',
    'dropout': 0.3,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'max_epochs': MAX_EPOCHS,
    'patience': PATIENCE,
    'validation_metrics': {
        'auc_roc': float(val_auc_roc),
        'auc_pr': float(val_auc_pr),
        'brier_score': float(val_brier),
        'dataset': 'Validation'
    },
    'test_metrics': {
        'auc_roc': float(test_auc_roc),
        'auc_pr': float(test_auc_pr),
        'brier_score': float(test_brier),
        'dataset': 'Test'
    }
}

with open(METRICS_FILE, 'w') as f:
    json.dump(all_metrics, f, indent=4)
print(f"Metrics saved to {METRICS_FILE}")

Model saved to ../models/mlp_model.pth
Predictions saved to ../results/mlp_predictions.csv
Metrics saved to ../results/mlp_metrics.json
